In [1]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(magrittr)
library(padr)
library(kableExtra)
library(purrr)
library(ggplot2)
library(ggthemr)
library(here)
library(lubridate)
# library(caret)
# library(scales)
ggthemr("fresh")
devtools::load_all()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘magrittr’


The following object is masked from ‘package:tidyr’:

    extract



Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows



Attaching package: ‘purrr’


The following object is masked from ‘package:magrittr’:

    set_names


here() starts at /users/sadamov/RProjects/CHAPo


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading CHAPo



In [ ]:
load(paste0(here(), "/data/other/species.RData"))
load(paste0(here(), "/data/other/stations.RData"))

In [ ]:


data_dwh <- import_data_dwh(paste0(here(), "/data/dwh/pollen_dwh_hourly.txt")) %>%
  aggregate_pollen() %>%
  impute_daily()
data_sdes <- import_data_cosmo("/scratch/sadamov/wd/20_poac_pheno_v2/mod_sdes_combined.txt", type = "Phenology V2") %>%
  aggregate_pollen() %>%
  impute_daily()
data_saisl <- import_data_cosmo("/scratch/sadamov/wd/20_poac_pheno_v2/mod_saisl_combined.txt", type = "Phenology V2") %>%
  aggregate_pollen() %>%
  impute_daily()


invisible(
data_dwh %<>% filter(
  between(date, min(data_sdes$date), max(data_sdes$date)),
  taxon == unique(data_sdes$taxon)
))

data_pheno <- data_sdes %>%
  select(date, station, sdes = value) %>%
  left_join(data_saisl %>%
    select(date, station, saisl = value),
    by = c("date", "station"))


In [ ]:
curves_pheno <- data_pheno %>%
  split(.$station) %>%
  map(~ .x %>%
    pmap(function(...) {
      current <- tibble(...)
      tibble(x = unique(data_pheno$date), value = current$sdes * current$saisl)
    }) %>%
    setNames(unique(data_pheno$date))
  )
str(curves_pheno)

In [ ]:
data_dwh %>%
  ggplot(aes(x = date)) +
  geom_col(aes(y = value)) +
  geom_line(data = data_pheno, aes(y = sdes), col = "red")